## LLM tipli AI

1. Openai- GPT4 1.5 trilyon
2. Claude (Anthropic)
3. LLAMA 1, 2
4. Falcon
5. Baba (Ali Baba)

Workflow
1. Read the file and create knowledgebase
2. Set up language model and chain
3. Write queries

In [22]:
import langchain
import openai #openai ilə işləmək üçün
from dotenv import load_dotenv
import os


from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory



load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



In [2]:
doc_loader = PyPDFLoader("pandasDocumentation.pdf")
doc = doc_loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 4000)
splitted_text = text_splitter.split_documents(doc)

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY,
                             model = "text-embedding-ada-002",
                             timeout = 50)

## To save embeddings locally Vectorstores

In [12]:
save_directory = "PandasDB"

db = Chroma.from_documents(documents= splitted_text,
                           embedding= embeddings,
                           persist_directory= save_directory
                          )

db.persist()

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-EblPKrcJWK3OmU3Axd0MLDaI on tokens per min. Limit: 1000000 / min. Current: 854062 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-EblPKrcJWK3OmU3Axd0MLDaI on tokens per min. Limit: 1000000 / min. Current: 753916 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-EblPKrcJWK3OmU3Axd0MLDaI on tokens per min. Limit: 

In [13]:
db = None

directory_to_load_from = "PandasDB"

db = Chroma(persist_directory= directory_to_load_from,
                            embedding_function= embeddings)


In [54]:
LLM = ChatOpenAI(openai_api_key = OPENAI_API_KEY,
                 temperature= 1
                )

memory = ConversationBufferMemory(llm = LLM,
                                  output_key= "answer",
                                  memory_key = "chat_history",
                                  return_messages= True
                                 )
retriever = db.as_retriever(k = 3)

chain = ConversationalRetrievalChain.from_llm(
                                     llm = LLM,
                                     memory = memory,
                                     verbose= False,
                                     retriever= retriever,
                                     return_source_documents= False,
                                     chain_type= "stuff",
                                     get_chat_history = lambda x: x
                                    )


In [37]:
x = chain.run({"question": """
            I have a pandas dataframe which I want to turn into numpy array.
            Write me an example code which does that. My dataframe has 250x400 shape"""}
         )

In [39]:
from IPython.display import Markdown, display

In [40]:
display(Markdown(x))

To convert a pandas DataFrame with a shape of 250x400 into a numpy array, you can use the `to_numpy()` method. Here is an example code:

```python
import pandas as pd

# Assuming you have a DataFrame called df with shape 250x400
numpy_array = df.to_numpy()

print(numpy_array.shape)  # Output: (250, 400)
```

This code will convert the DataFrame `df` into a numpy array `numpy_array` with the same shape.

In [45]:
def display_markdown(func):

    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        display(Markdown(result))
        return result
    return wrapper

class MyMarkdownBasedChain(ConversationalRetrievalChain):

    def run(self, *args, **kwargs):
        original_result = super().run(*args, **kwargs)
        return display(Markdown(original_result))


    

In [46]:
chain_2 = MyMarkdownBasedChain.from_llm(
                                     llm = LLM,
                                     memory = memory,
                                     verbose= False,
                                     retriever= retriever,
                                     return_source_documents= False,
                                     chain_type= "stuff",
                                     get_chat_history = lambda x: x
                                    )


In [47]:
chain_2.run({"question": """
            I have a pandas dataframe which I want to turn into numpy array.
            Write me an example code which does that. My dataframe has 250x400 shape"""}
         )

To convert a pandas DataFrame with a shape of 250x400 into a numpy array, you can use the `to_numpy()` method. Here is an example code:

```python
import pandas as pd
import numpy as np

# Create a sample DataFrame
df = pd.DataFrame(np.random.randn(250, 400))

# Convert DataFrame to numpy array
array = df.to_numpy()

# Print the shape of the numpy array
print(array.shape)
```

This code will convert the DataFrame `df` into a numpy array `array` and then print the shape of the array, which should be (250, 400).

In [49]:
chain_2.run("""
            How to solve a KeyError problem in Pandas


           Gender   Value   Year    Measure
Name                
Ma         girl     0       1997    Rank
Nabiha     girl     1204    2014    Rank
Muad.      boy      1749    2009    Rank
Zynah      girl     4763    2017    Rank
Aathavi    girl     0       2019    Rank
Kia-Marie  girl     0       2017    Count
Tishaan    boy      0       2006    Rank
Ryyan      boy      0       2010    Count
Llyr       boy      1852    2012    Rank
Abdirazaq  boy      0       2017    Rank

given this dataframe, when i try to apply a custom function to this dataframe as seen
below, i get keyError. It complains about Value.

df['Rank'] = df.apply(adjust_rank)
Why do I get: KeyError: 'Value'?
"""
           )

Based on the given context, there is no specific information about the 'Value' column in the DataFrame. Therefore, it is likely that the KeyError is occurring because the 'Value' column does not exist in the DataFrame.

In [50]:
chain_2.run("""
            It does exist in the dataframe though
"""
           )

I don't have enough information to answer that question.

In [53]:
chain_2.run("""
                give me an example code on pandas pivot. generate a python script on pandas pivot
"""
           )

Certainly! Here's an example code on pandas pivot:

```python
import pandas as pd

# Create a sample DataFrame
data = {
    'City': ['New York', 'Chicago', 'Los Angeles', 'New York', 'Chicago', 'Los Angeles'],
    'Year': [2019, 2019, 2019, 2020, 2020, 2020],
    'Sales': [100, 200, 150, 120, 180, 130]
}

df = pd.DataFrame(data)

# Pivot the DataFrame
pivot_df = df.pivot(index='City', columns='Year', values='Sales')

print(pivot_df)
```

This code creates a DataFrame with columns 'City', 'Year', and 'Sales'. It then uses the `pivot()` function to pivot the DataFrame, with 'City' as the index, 'Year' as the columns, and 'Sales' as the values. The resulting pivoted DataFrame is then printed.

And here's the equivalent Python script:

```python
import pandas as pd

data = {
    'City': ['New York', 'Chicago', 'Los Angeles', 'New York', 'Chicago', 'Los Angeles'],
    'Year': [2019, 2019, 2019, 2020, 2020, 2020],
    'Sales': [100, 200, 150, 120, 180, 130]
}

df = pd.DataFrame(data)

pivot_df = df.pivot(index='City', columns='Year', values='Sales')

print(pivot_df)
```

You can save this script in a Python file (e.g., `pivot_example.py`) and run it using a Python interpreter or IDE.

In [56]:
chain_2.run(""" given your example above, explain it step by step""")

Sure! Let's go through the example code step by step:

1. First, we import the pandas library and numpy library:
   ```python
   import pandas as pd
   import numpy as np
   ```

2. Next, we create a list of tuples called "tuples". Each tuple contains two elements: the first element is a string representing a category ("bar", "baz", "foo", "qux"), and the second element is a string representing a sub-category ("one", "two"):
   ```python
   tuples = list(zip(*[["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"], ["one", "two", "one", "two", "one", "two", "one", "two"]]))
   ```

3. We create a MultiIndex from the "tuples" list, with the names "first" and "second" for the levels of the index:
   ```python
   index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
   ```

4. We create a DataFrame called "df" with random values from a normal distribution, with 8 rows and 2 columns. We set the index of the DataFrame to be the "index" MultiIndex we created earlier, and we name the columns "A" and "B":
   ```python
   df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
   ```

5. We create a new DataFrame called "df2" by selecting the first 4 rows of the "df" DataFrame:
   ```python
   df2 = df[:4]
   ```

6. Finally, we display the "df2" DataFrame:
   ```python
   df2
   ```

The output will be a DataFrame with 4 rows and 2 columns, where the rows are labeled with the categories and sub-categories from the "tuples" list, and the columns are labeled "A" and "B". The values in the DataFrame are random numbers from a normal distribution.